In [ ]:
%ls

drive/  sample_data/


In [ ]:
%cd /content/drive/MyDrive/projeto-final-vamoai

/content/drive/MyDrive/projeto-final-vamoai


### Importando bibliotecas necessárias para criação e modificações no Dataframe

In [ ]:
import pandas as pd
import json
import csv

In [ ]:
from pandas import json_normalize
from pathlib import Path
from datetime import datetime

### Criando a conexão com o arquivo no google drive e criando um dataframe

In [ ]:
p = Path(r'/content/drive/MyDrive/projeto-final-vamoai/BrowserHistory.json')
with p.open('r', encoding='utf-8') as f:
    data = json.loads(f.read())
df = pd.json_normalize(data,'Browser History')


### Criando conexão com o segundo Dataframe

In [ ]:
p = Path(r'/content/drive/MyDrive/projeto-final-vamoai/BrowserHistory_anterior_2020.json')
with p.open('r', encoding='utf-8') as f:
    data = json.loads(f.read())
segundo_df = pd.json_normalize(data,'Browser History')

### Unindo os dataframes

In [ ]:
historico_google = pd.concat([df, segundo_df], axis=1)

### Consultando as informações do dataframe para identificar os tipos de dados

In [ ]:
historico_google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43499 entries, 0 to 43498
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   favicon_url      37478 non-null  object 
 1   page_transition  43499 non-null  object 
 2   title            43499 non-null  object 
 3   url              43499 non-null  object 
 4   client_id        43499 non-null  object 
 5   time_usec        43499 non-null  int64  
 6   favicon_url      9972 non-null   object 
 7   page_transition  13069 non-null  object 
 8   title            13069 non-null  object 
 9   url              13069 non-null  object 
 10  client_id        13069 non-null  object 
 11  time_usec        13069 non-null  float64
dtypes: float64(1), int64(1), object(10)
memory usage: 4.0+ MB


### Renomeando as colunas com o mesmo nome para unir as colunas e linhas dos 2 Dataframes

In [ ]:
relacao_colunas = {'title':'title','url':'url','page_transition':'page_transition','client_id':'client_id',
                   'time_usec':'time_usec','favicon_url':'favicon_url'}
historico_google = pd.concat([df, segundo_df.rename(columns=relacao_colunas)])

In [ ]:
historico_google

,favicon_url,page_transition,title,url,client_id,time_usec
0,https://www.youtube.com/s/desktop/a910c60b/img...,TYPED,(381) YouTube,https://www.youtube.com/,C785xo8eVcJWEPtQtbFwgg==,1626544173303193
1,https://www.google.com/favicon.ico,LINK,Google Takeout,https://takeout.google.com/settings/takeout?pli=1,C785xo8eVcJWEPtQtbFwgg==,1626544112245772
2,https://exame.com/wp-content/uploads/2020/05/f...,LINK,Como baixar o relatório de dados que o Google ...,https://exame.com/tecnologia/como-baixar-o-rel...,C785xo8eVcJWEPtQtbFwgg==,1626544096209782
3,https://support.google.com/favicon.ico,LINK,Informar erros de dados ou conteúdo no Google ...,https://support.google.com/maps/answer/3094088...,C785xo8eVcJWEPtQtbFwgg==,1626543989601060
4,https://support.google.com/favicon.ico,LINK,Solicitar a propriedade do Perfil de uma empre...,https://support.google.com/business/answer/456...,C785xo8eVcJWEPtQtbFwgg==,1626543981610592
...,...,...,...,...,...,...
13064,NaN,FORM_SUBMIT,gesso campo grande rj - Pesquisa Google,https://www.google.com/search?client=ms-androi...,9JwocHYFXsmD0/3fw3LPIw==,1595686962308307
13065,NaN,LINK,beverly - Pesquisa Google,https://www.google.com/search?q=beverly&oq=bev...,9JwocHYFXsmD0/3fw3LPIw==,1595686954998421
13066,NaN,RELOAD,beverly - Pesquisa Google,https://www.google.com/search?q=beverly&oq=bev...,9JwocHYFXsmD0/3fw3LPIw==,1595686947761163
13067,NaN,GENERATED,beverly - Pesquisa Google,https://www.google.com/search?q=beverly&oq=bev...,9JwocHYFXsmD0/3fw3LPIw==,1595643040344252


### Apagando as colunas que não desejo utilizar

In [ ]:
historico_google.drop(columns=['favicon_url','page_transition','client_id'], inplace=True )

### Apagando valores nulos do Dataframe

In [ ]:
historico_google.dropna(inplace=True)

In [ ]:
historico_google

,title,url,time_usec
0,(381) YouTube,https://www.youtube.com/,1626544173303193
1,Google Takeout,https://takeout.google.com/settings/takeout?pli=1,1626544112245772
2,Como baixar o relatório de dados que o Google ...,https://exame.com/tecnologia/como-baixar-o-rel...,1626544096209782
3,Informar erros de dados ou conteúdo no Google ...,https://support.google.com/maps/answer/3094088...,1626543989601060
4,Solicitar a propriedade do Perfil de uma empre...,https://support.google.com/business/answer/456...,1626543981610592
...,...,...,...
13064,gesso campo grande rj - Pesquisa Google,https://www.google.com/search?client=ms-androi...,1595686962308307
13065,beverly - Pesquisa Google,https://www.google.com/search?q=beverly&oq=bev...,1595686954998421
13066,beverly - Pesquisa Google,https://www.google.com/search?q=beverly&oq=bev...,1595686947761163
13067,beverly - Pesquisa Google,https://www.google.com/search?q=beverly&oq=bev...,1595643040344252


### Transformando dados do tipo int em Datetime

In [ ]:
historico_google['time_usec'] = historico_google['time_usec'].apply(lambda x : pd.to_datetime(datetime.fromtimestamp(x/1e6).strftime('%d/%m/%Y'),unit='ns', errors='ignore'))

### Confirmando se alterações foram realizadas com sucesso!

In [ ]:
historico_google.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56568 entries, 0 to 13068
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   title      56568 non-null  object        
 1   url        56568 non-null  object        
 2   time_usec  56568 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 1.7+ MB


### Renomeando colunas do Dataframe

In [ ]:
historico_google.rename(columns={'title':'titulo','url':'url','time_usec':'data'},inplace=True)

### Confirmando as alterações realizadas

In [ ]:
historico_google

,titulo,url,data
0,(381) YouTube,https://www.youtube.com/,2021-07-17
1,Google Takeout,https://takeout.google.com/settings/takeout?pli=1,2021-07-17
2,Como baixar o relatório de dados que o Google ...,https://exame.com/tecnologia/como-baixar-o-rel...,2021-07-17
3,Informar erros de dados ou conteúdo no Google ...,https://support.google.com/maps/answer/3094088...,2021-07-17
4,Solicitar a propriedade do Perfil de uma empre...,https://support.google.com/business/answer/456...,2021-07-17
...,...,...,...
13064,gesso campo grande rj - Pesquisa Google,https://www.google.com/search?client=ms-androi...,2020-07-25
13065,beverly - Pesquisa Google,https://www.google.com/search?q=beverly&oq=bev...,2020-07-25
13066,beverly - Pesquisa Google,https://www.google.com/search?q=beverly&oq=bev...,2020-07-25
13067,beverly - Pesquisa Google,https://www.google.com/search?q=beverly&oq=bev...,2020-07-25


### Apagando termos específicos do Dataframe

In [ ]:
historico_google['titulo']=historico_google['titulo'].map(lambda x: x.lstrip(' '))

In [ ]:
historico_google['titulo']=historico_google['titulo'].map(lambda x: x.lstrip('Redirect'))

### Realizando contagem de acesso organizado por meio do mais acessado ao menos acessado

In [ ]:
historico_google['titulo'].value_counts()

Nova guia                                                                                             4651
globo.com - Absolutamente tudo sobre notícias, esportes e entretenimento                              2842
                                                                                                      2360
Play Kahoot! - Enter game PIN here!                                                                    511
Google                                                                                                 485
                                                                                                      ... 
(80) oracle developer - YouTube                                                                          1
Museus Castro Maya                                                                                       1
Como fazer a consulta e o registro da inscrição estadual para o MEI | Portal de Atendimento SEBRAE       1
assistir online fenomenos paranormais

### Realizando pesquisa de termos específicos do Dataframe

In [ ]:
pesquisas_sobre_dados = historico_google.query('titulo.str.contains("Python") | titulo.str.contains("Dados") | titulo.str.contains("Análise")\
| titulo.str.contains("Programação") |titulo.str.contains("Data") |titulo.str.contains("analysis")', engine='python')

### Criando novo Dataframe para pesquisar termos específicos em intervalo de tempo

In [ ]:
antes_do_vamoai = pesquisas_sobre_dados.loc[(pesquisas_sobre_dados['data'] > '2020-07-01') & (pesquisas_sobre_dados['data'] < '2021-01-25')]
antes_do_vamoai['titulo'].value_counts()

Merchant Portal | First Data                                                                               26
Governança e Qualidade de Dados: aulas online para Enem e Vestibulares | Descomplica                        7
Welcome to Python.org                                                                                       7
Aula 2 - Para que serve o Python? - Curso em Vídeo                                                          7
Exercícios - Implantando a Governança de Dados                                                              7
                                                                                                           ..
Confirmação Necessária - Intensivão de Python - marianlimadocarmo@gmail.com - Gmail                         1
(175) Exercício Python #011 - Pintando Parede - YouTube                                                     1
Exercicios-Python/008 - Conversor de Medidas.py at master · andreinaoliveira/Exercicios-Python · GitHub     1
Eu fiz um 

### Realizando consulta de termos específicos em período de tempo específico

In [ ]:
terceiro_modulo = pesquisas_sobre_dados.loc[(pesquisas_sobre_dados['data'] > '2021-04-12') & (pesquisas_sobre_dados['data'] < '2021-06-04')]
terceiro_modulo['titulo'].value_counts()

Dados Prestador Médico - vanessacarolinamelodlima@gmail.com - Gmail                                     18
serenozin/DataVizu_VamoAI                                                                               14
DataVizu_VamoAI/perguntas-projeto-lol.sql at main · serenozin/DataVizu_VamoAI                            6
Python 3 parte 1: Aula 3 - Atividade 1 A condição elif | Alura - Cursos online de tecnologia             6
Dados do pedido                                                                                          5
                                                                                                        ..
Criando ambientes virtuais para projetos Python com o Virtualenv - Blog da TreinaWeb                     1
Food oferece curso gratuito em Ciência de Dados para pessoas de baixa renda - Canaltech                  1
(198) Introdução à Programação com Python - Parte 1 - YouTube                                            1
Python 3 parte 1: Aula 2 - Atividade 

### Transformando o Dataframe em arquivo csv

In [ ]:
historico_google.to_csv ('historico_pesquisa_google.csv',index=False)

In [ ]:
from google.colab import files
files.download("historico_pesquisa_google.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>